# Google Brain - Ventilator Pressure Predictio

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.callbacks import TensorBoard

In [2]:
train = pd.read_csv("train.csv")
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


In [4]:
train = train.sample(frac =.01,random_state = 90, replace = False)
train.shape

(60360, 8)

In [5]:
train.isnull().sum()


id           0
breath_id    0
R            0
C            0
time_step    0
u_in         0
u_out        0
pressure     0
dtype: int64

In [6]:
train = train.drop("breath_id",axis=1)

In [7]:
x = train.drop("pressure",axis=1)
y = train["pressure"]

In [8]:
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=[6]),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='selu'),
    layers.Dense(1),
])
model.compile(optimizer='adam',loss='mae',)

2022-09-03 15:11:24.682280: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
history = model.fit(x, y,validation_data=(x, y),batch_size=256,epochs=10,)

Epoch 1/10
236/236 [==============================] - 4s 17ms/step - loss: 17359.3965 - val_loss: 2406.9221
Epoch 2/10
236/236 [==============================] - 4s 17ms/step - loss: 453.5638 - val_loss: 662.5531
Epoch 3/10
236/236 [==============================] - 4s 17ms/step - loss: 107.9351 - val_loss: 15.6893
Epoch 4/10
236/236 [==============================] - 4s 18ms/step - loss: 34.3098 - val_loss: 5.0749
Epoch 5/10
236/236 [==============================] - 4s 17ms/step - loss: 46.1375 - val_loss: 5.0725
Epoch 6/10
236/236 [==============================] - 5s 19ms/step - loss: 5.0724 - val_loss: 5.0720
Epoch 7/10
236/236 [==============================] - 4s 18ms/step - loss: 81.1985 - val_loss: 5.0833
Epoch 8/10
236/236 [==============================] - 6s 23ms/step - loss: 41.2243 - val_loss: 5.0725
Epoch 9/10
236/236 [==============================] - 5s 22ms/step - loss: 5.0726 - val_loss: 5.0724
Epoch 10/10
236/236 [==============================] - 6s 25ms/step - los

In [10]:
model.save("saved_model")


INFO:tensorflow:Assets written to: saved_model/assets


In [11]:
loaded_model = keras.models.load_model("./saved_model")


In [13]:
preds = loaded_model.predict(x[:1000])
mae = mean_absolute_error(y[:1000],preds)
print(mae)

32/32 [==============================] - 0s 2ms/step
5.129622693308366
